In [6]:
pip install pyaudio

  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp38-cp38-macosx_10_9_x86_64.whl size=23491 sha256=44f53baa4ff9eb6430eec1deee83f0e10e17181ce9b86d7b12f544774c792a97
  Stored in directory: /Users/home/Library/Caches/pip/wheels/cf/be/f6/06f4d9fb96713a3ba58a480dd5739b3bf66ff0430de58a38c1
Successfully built pyaudio
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyaudio
import wave
import time
from IPython.display import clear_output, display
 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 512
RECORD_SECONDS = 420
WAVE_OUTPUT_FILENAME = "recordedFile"
device_index = 2

In [2]:
# List and choose input sources
audio = pyaudio.PyAudio()
info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
inputs = [False] * numdevices
while (True):
    print("Enter device id to select, and enter something random to finish")
    for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("✅" if inputs[i] else "❌", "Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))
    try:
        index = int(input())
        if (index < 0 or index >= numdevices):
            break
        inputs[index] = True
    except ValueError:
        break
    clear_output(wait=True)

Enter device id to select, and enter something random to finish
❌ Input Device id  1  -  HD Web Camera
❌ Input Device id  2  -  BlackHole 2ch
✅ Input Device id  3  -  External Microphone
✅ Input Device id  5  -  MacBook Pro Microphone
✅ Input Device id  7  -  James’s AirPods Pro
❌ Input Device id  10  -  ZoomAudioDevice


In [3]:
# Record audio directly to file

# Start recording
streams = []
for index in range(numdevices):
    if inputs[index]:
        streams.append(audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, input_device_index = index, frames_per_buffer=CHUNK))
print("Recording started with ", len(streams), " devices")

# Open files
waveFiles = []
for i in range(len(streams)):
    waveFiles.append(wave.open(WAVE_OUTPUT_FILENAME+"-"+str(i)+".wav", 'wb'))
    waveFiles[i].setnchannels(CHANNELS)
    waveFiles[i].setsampwidth(audio.get_sample_size(FORMAT))
    waveFiles[i].setframerate(RATE)
print("Opened wav files")

# Write to files
startTime = time.time()
dh = display('Time elapsed: 0s', display_id=True)
for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
    frames = [ [] for _ in range(len(streams)) ]
    for j in range(len(streams)):
        data = streams[j].read(CHUNK, exception_on_overflow = False)
        frames[j].append(data)
        waveFiles[j].writeframes(b''.join(frames[j]))
    dh.update("Time elapsed: " + str(int(time.time() - startTime)) + "s")
    
# Stop recording
for stream in streams:
    stream.stop_stream()
    stream.close()
audio.terminate()
print ("recording finished. You can either re-record or go to the next cell to save the file")

# Save files
for i in range(len(streams)):
    waveFiles[i].close()
print("Files saved")

Recording started with  3  devices
Opened wav files


'Time elapsed: 0s'

recording finished. You can either re-record or go to the next cell to save the file
Files saved


In [3]:
# Record audio

streams = []
for index in range(numdevices):
    if inputs[index]:
        streams.append(audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, input_device_index = index, frames_per_buffer=CHUNK))

print("Recording started with ", len(streams), " devices")

frames = [ [] for _ in range(len(streams)) ]

for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
    for i in range(len(streams)):
        data = streams[i].read(CHUNK, exception_on_overflow = False)
        frames[i].append(data)

for stream in streams:
    stream.stop_stream()
    stream.close()
audio.terminate()

print ("recording finished. You can either re-record or go to the next cell to save the file")

Recording started with  3  devices
recording finished. You can either re-record or go to the next cell to save the file


In [4]:
# Save to files

for i in range(len(frames)):
    waveFile = wave.open(WAVE_OUTPUT_FILENAME+"-"+str(i)+".wav", 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames[i]))
    waveFile.close()
del frames

In [6]:
print(frames)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

